In [1]:
# Used for reading the book files
import os

# Used for cleaning the book text
import re

## Task 1: Third-order letter approximation model
Create a trigram model from the given books

In [2]:
def create_trigram_model():

    BOOK_DIRECTORY_PATH = "./books"

    # Read in all of the text from the supplied books as training data
    book_text = read_in_files_from_directory(BOOK_DIRECTORY_PATH)

    # Remove unwanted characters from the text and convert to uppercase
    book_text = cleaned_book_text = clean_text(book_text)

    # Print first 500 characters as a test
    print(book_text[0:500])

In [3]:
# Read all files in a given directory
def read_in_files_from_directory(directory_name):
    
    # Get the file names the books
    book_list = os.listdir(directory_name)
    text = ""

    print("Files and directories in '", directory_name, "' :", book_list)

    # Read in all of the text from the books
    for book in book_list:
        # Open the current book
        f = open(directory_name + "/" + book, "r", encoding="utf8")

        # Read the contents of the current book
        current_book_text = f.read()

        # Remove the preamble and postamble from the current book
        current_book_text = remove_preabmle_and_postamble(current_book_text)
        
        text += current_book_text + "\n"
        f.close()
        
    return text

In [4]:
# Remove preabmle and postamble from given book text
def remove_preabmle_and_postamble(text):

    end_of_preamble_string = "*\n"
    end_of_postamble_string = "\n*"

    # Remove preamble and postabmle
    # https://stackoverflow.com/a/59903231
    text = text[text.find(end_of_preamble_string):text.rfind(end_of_postamble_string)]

    # Remove "*" from start and end of trimmed text
    text = text[1:-1]

    # Remove blank lines from start and end of from trimmed text
    text = text.strip()

    # Test if the preamble and post amble were removed correctly
    # print(text[0:500], "\n=============\n")
    # print(text[len(text) - 500:len(text)], "\n=============\n")

    return text

In [5]:
# Remove unwanted characters from a given string
def clean_text(text):

    # Remove unwanted characters from the text
    cleaned_text = re.sub(r'[^a-zA-Z\s.]', '', text)
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("\n", " ")

    # Convert all characters to uppercase
    cleaned_text = cleaned_text.upper()
    
    return cleaned_text

## Run the program

In [6]:
create_trigram_model()

Files and directories in ' ./books ' : ['Frankenstein.txt', 'LittleWomen.txt', 'MiddleMarch.txt', 'MobyDick.txt', 'PrideAndPrejudice.txt']
FRANKENSTEIN  OR THE MODERN PROMETHEUS  BY MARY WOLLSTONECRAFT GODWIN SHELLEY    CONTENTS   LETTER   LETTER   LETTER   LETTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER   CHAPTER      LETTER   TO MRS. SAVILLE ENGLAND.   ST. PETERSBURGH DEC. TH .   YOU WILL REJOICE TO HEAR THAT NO DISASTER HAS ACCOMPANIED THE COM
